In [ ]:
# =============================================================================
# =============================================================================
# # 문제 11 유형(DataSet_11.csv 이용)

# 구분자 : comma(“,”), 470 Rows, 4 Columns, UTF-8 인코딩

# 세계 각국의 행복지수를 비롯한 여러 정보를 조사한 DS리서치는
# 취합된 자료의 현황 파악 및 간단한 통계분석을 실시하고자 한다.

# 컬 럼 / 정 의 / Type
# Country / 국가명 / String
# Happiness_Rank / 당해 행복점수 순위 / Double
# Happiness_Score / 행복점수 / Double
# year / 년도 / Double
# =============================================================================
# =============================================================================
# (참고)
# from statsmodels.formula.api import ols
# from statsmodels.stats.anova import anova_lm

In [1]:
import pandas as pd
import numpy as np

dataset11 = pd.read_csv('Dataset/Dataset_11.csv')

In [3]:
dataset11.head()

,Country,Happiness_Rank,Happiness_Score,year
0,Switzerland,1,7.587,2015
1,Iceland,2,7.561,2015
2,Denmark,3,7.527,2015
3,Norway,4,7.522,2015
4,Canada,5,7.427,2015


In [ ]:
#%%

# =============================================================================
# 1.분석을 위해 3년 연속 행복지수가 기록된 국가의 데이터를 사용하고자 한다. 
# 3년 연속 데이터가 기록되지 않은 국가의 개수는?
# - 국가명 표기가 한 글자라도 다른 경우 다른 국가로 처리하시오.
# - 3년 연속 데이터가 기록되지 않은 국가 데이터는 제외하고 이를 향후 분석에서
# 활용하시오.(답안 예시) 1
# =============================================================================

In [57]:
q1 = dataset11.copy()

In [58]:
q1.year.unique()

array([2015, 2016, 2017], dtype=int64)

In [59]:
q1_2 = q1.pivot_table(index='Country', columns='year', values = 'Happiness_Score')

In [56]:
q1_2['na_num'] = q1_2.isnull().sum(axis=1)

In [29]:
(q1_2['na_num']>0).sum()

20

In [ ]:
#%%

# =============================================================================
# 2.(1번 산출물을 활용하여) 2017년 행복지수와 2015년 행복지수를 활용하여 국가별
# 행복지수 증감률을 산출하고 행복지수 증감률이 가장 높은 3개 국가를 행복지수가
# 높은 순서대로 차례대로 기술하시오.
# 증감률 = (2017년행복지수−2015년행복지수)/2
# 
# - 연도는 년월(YEAR_MONTH) 변수로부터 추출하며, 연도별 매출금액합계는 1월부터
# 12월까지의 매출 총액을 의미한다. (답안 예시) Korea, Japan, China
# =============================================================================

In [31]:
q2 = q1_3.drop(columns='na_num')

In [34]:
# 증감률
q2['ratio'] = (q2[2017] - q2[2015]) /2

In [37]:
q2['ratio'].nlargest(3).index
# 정답 : 'Latvia', 'Romania', 'Togo'

Index(['Latvia', 'Romania', 'Togo'], dtype='object', name='Country')

In [38]:
#%%

# =============================================================================
# 3.(1번 산출물을 활용하여) 년도별 행복지수 평균이 유의미하게 차이가 나는지
# 알아보고자 한다. 
# 이와 관련하여 적절한 검정을 사용하고 검정통계량을 기술하시오.
# - 해당 검정의 검정통계량은 자유도가 2인 F 분포를 따른다.
# - 검정통계량은 소수점 넷째 자리까지 기술한다. (답안 예시) 0.1234
# =============================================================================


In [39]:
from statsmodels.formula.api import ols
from statsmodels.stats.anova import anova_lm

In [44]:
q3 = dataset11[dataset11['Country'].isin(q1_2.index.values)]

In [40]:
# C(수치형변수) : 범주형으로 인식
anova = ols('Happiness_Score~C(year)',q1).fit()
# ols 사용 시 변수명에서 스페이스가 포함된 변수, -포함된 변수, .포함된 변수 사용

In [46]:
anova_lm(anova)
## C(year) 집단간
## Residual 집단내
# 적어도 한집단이 차이가 있다.

# 정답 : 0.026129

,df,sum_sq,mean_sq,F,PR(>F)
C(year),2.0,0.067840,0.033920,0.026129,0.974211
Residual,467.0,606.238796,1.298156,NaN,NaN


In [50]:
from statsmodels.stats.multicomp import pairwise_tukeyhsd

q3_out = pairwise_tukeyhsd(q3['Happiness_Score'], q3['year'])

In [52]:
# 집단간 차이 검정
print(q3_out)

Multiple Comparison of Means - Tukey HSD, FWER=0.05
group1 group2 meandiff p-adj  lower  upper  reject
--------------------------------------------------
  2015   2016   0.0065   0.9 -0.2954 0.3083  False
  2015   2017  -0.0217   0.9 -0.3246 0.2811  False
  2016   2017  -0.0282   0.9 -0.3315 0.2752  False
--------------------------------------------------


In [62]:
from scipy.stats import f_oneway

f_oneway(q1_2[2015].values, q1_2[2016].values, q1_2[2017].values)

F_onewayResult(statistic=nan, pvalue=nan)